## Import Data

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np

# Read Excel files
data = "Marshall_Course_Enrollment_1516_1617.xlsx"
data2 = "Marshall_Room_Capacity_Chart.xlsx"
# Read sheets needed
schedules = pd.read_excel(data, sheet_name="Schedules")
room = pd.read_excel(data2, sheet_name="Room", index_col=0)

## Define a function to convert time

In [2]:
# Convert time into decimal format in hours
def convert(inputTime):
    try:
        hh, mm, ss = str(inputTime).split(':')
        ans = (float(hh)+float(mm)/float(60))
    except:
        ans = 0
    return ans

# Define a function to calculate the number of days each course requires

In [3]:
def n_o_d(Days):
    try:
        ans = len(Days)
    except:
        ans = 9999
    return ans

## Convert the original data

In [4]:
# Apply the convert function to begin time
schedules['First Begin Time'] = schedules['First Begin Time'].apply(convert)

# Apply the convert function to end time
schedules['First End Time'] = schedules['First End Time'].apply(convert)

# Calculate time slots needed for each course
schedules['Slots'] = np.ceil((schedules['First End Time'] - schedules['First Begin Time'])/2)

# Calculate number of days each course requires
schedules['Days per week'] = schedules['First Days'].apply(n_o_d)

## Data cleaning

In [5]:
# Filter data for only courses in 2017 spring
schedules = schedules[schedules['Term'] == 20171]

# List of marshall departments
dept = ["ACCT", "BAEP", "BUCO", "DSO", "FBE", "MKT", "MOR"]

# Filter data based on the list
schedules = schedules[schedules['Department'].isin(dept)]

# Exclude online courses and courses in office
schedules = schedules[schedules['First Room'] != 'ONLINE']
schedules = schedules[schedules['First Room'] != 'OFFICE']

# Exclude courses with too large registered count that marshall classrooms cannot accommodate
schedules = schedules[schedules['Reg Count'] <= max(room.loc[:,'Size'])]

# Exclude courses with unknown duration
schedules = schedules[schedules['Slots'] != 0]

# Exclude courses that require more than two hours per day
schedules = schedules[schedules['Slots'] <= 1]

# Exclude courses that require more than two days per week
schedules = schedules[schedules['Days per week'] <= 2]

# Reindex schedules
schedules.index = range(0,len(schedules))

## Before Optimization: Average Seat Utilization Rate 

In [6]:
# Join datasets with key 'First Room' for schedules and key 'Room' for room
data = (pd.merge(schedules, room, how = 'left', left_on= 'First Room', right_index=True)
       [['Term','Section','Department','First Begin Time','First End Time','First Room',
         'Reg Count','Seats','Size']]
       ) # Size is from room dataset not schedules dataset

# Filter out courses held in rooms outside Marshall
data = data[data['Size'].notnull()]

# Define a dictionary for the dataframe aggregation
aggregations = {
    'Registered':'sum',
    'Size':'sum'
}
data = data.rename(columns={'Reg Count':'Registered'})


output = data.agg(aggregations)
print('Average seat utilization rate in 20171: {0:.2f}' .format(output.Registered/output.Size))

Average seat utilization rate in 20171: 0.67


## Before Optimization: Percentage of Students Taking Courses in Prime Time 

In [7]:
# Filter out courses held NOT in Prime Time
primeData = data.drop(data[(data['First Begin Time'] < float(10)) | (data['First End Time'] > float(16))].index)


outputPrime = primeData.agg(aggregations)
print('Percentage of students taking courses in prime time in 20171: {0:.2f}' 
      .format(outputPrime.Registered/output.Registered))

Percentage of students taking courses in prime time in 20171: 0.59


## Input data

In [8]:
I = ['M','T','W','H','F'] # day of week
J = np.arange(8, 22, 2) # time slot
Jprime = np.arange(10, 16, 2) # time slot in prime time
K = room.index # classroom

# course (with different sections)
Z = [] 
for z in range(0,len(schedules.index)):
    Z.append(schedules.loc[z, 'Course'] + ' ' + str(schedules.loc[z, 'Section'])) 

# reindex by using the combined course name and section so that each index is unique
schedules.index = Z

# registered count of each course
r = {} 
for z in Z:
    r[z] = schedules.loc[z, 'Reg Count']

# number of days each course requires
n = {} 
for z in Z:
    n[z] = schedules.loc[z, 'Days per week']

# size of classroom
s = {} 
for k in K:
    s[k] = room.loc[k, 'Size']

## Build a model

In [9]:
# Our decision variable x[i,j,k,z] is a binary variable of
# whether course z is assigned to ith day of week and jth time slot in classroom k
mod=grb.Model()
x={}
for i in I:
    for j in J:
        for k in K:
            for z in Z:
                x[i,j,k,z] = mod.addVar(vtype = grb.GRB.BINARY, name = 'x[{0},{1},{2},{3}]'.format(i, j, k, z))

## Objective function

In [10]:
# Our objective is to maximize the average seat utilization rate of all the courses offered by Marshall and at the same time
# maximize the percentage of students that take courses in the prime time (10:00-16:00)
# we assign equal weight (0.5,0.5) to both ratio in our final objective value
mod.setObjective(0.5 * ((sum((x[i,j,k,z] / n[z]) * (r[z] / s[k]) for i in I for j in J for k in K for z in Z))/len(Z)) +
                 0.5 * ((sum((x[i,j,k,z] / n[z]) * r[z] for i in I for j in Jprime for k in K for z in Z)) / 
                        (sum(r[z] for z in Z))), sense = grb.GRB.MAXIMIZE)

## Constraints:Classroom capacity

In [11]:
# The size of the classroom assigned should be greater or equal to the registered count of the course
for i in I:
    for j in J:
        for k in K:
            for z in Z:
                mod.addConstr(x[i,j,k,z] * (s[k] - r[z]) >= 0, 
                name = 'Number of seats in {0} is greater or equal to number of registered students in {1}'.format(k, z))

## Constraints:Classroom availability

In [12]:
# Each classroom in each time slot in each day of week can only be assigned no more than once to a course
for i in I:
    for j in J:
        for k in K:
            mod.addConstr(sum(x[i,j,k,z] for z in Z) <= 1, 
                name = 'Classroom {0} on {1} in time slot {2} is assigned to no more than one class'.format(k, i, j))

## Constraints:Course duration

In [13]:
# Each course z should be assigned to exact n[z] time slots
for z in Z:
    mod.addConstr(sum(x[i,j,k,z] for i in I for j in J for k in K) == n[z], 
                name = 'Course {0} is assigned to exact 1 time slot'.format(z))

## Constraint:Courses taught in two days

In [14]:
# For those courses having classes two days per week, if they are assigned to a time slot in a classroom in a day,
# they will be assigned to the same time slot in the same classroom two days later.
# And for those courses with two classes per week, they can't be assigned to Friday
for z in Z:
    if n[z] == 2:
        for j in J:
            for k in K:            
                mod.addConstr(x['M',j,k,z] == x['W',j,k,z])
                mod.addConstr(x['T',j,k,z] == x['H',j,k,z])
                mod.addConstr(x['F',j,k,z] == 0)

## Solve the Output

In [15]:
mod.setParam('OutputFlag',False)
mod.optimize()

# Calculate the average seat utilization rate
avg_utilization_rate = (sum((x[i,j,k,z].x / n[z]) * (r[z] / s[k]) for i in I for j in J for k in K for z in Z))/len(Z)
# Calculate percentage of students taking courses in prime time
percent_prime = (sum((x[i,j,k,z].x / n[z]) * r[z] for i in I for j in Jprime for k in K for z in Z)) / (sum(r[z] for z in Z))

# Building a dataframe for the output
Objective_Value = ['Optimal Objective', '{0:.2f}'.format(mod.ObjVal)]
Output_rate = ['Average seat utilization rate', '{0:.2f}'.format(avg_utilization_rate)]
Output_percentage = ['Percentage of students taking courses in prime time', '{0:.2f}'.format(percent_prime)]
Summary = pd.DataFrame([Objective_Value, Output_rate, Output_percentage], columns = ['', 'Value'])
Summary

,,Value
0,Optimal Objective,0.89
1,Average seat utilization rate,0.86
2,Percentage of students taking courses in prime...,0.92


## Solution

In [16]:
# Building a table via a list of lists
SolutionTable = []
for z in Z:
    for k in K:
        for i in I:
            for j in J:
                SolutionTable.append([z, k, i+' '+str(j), r[z], s[k], n[z], x[i,j,k,z].x])
# Transforming table to data frame
Solution = pd.DataFrame(SolutionTable, columns = ['Course', 'Classroom', 'Time', 'Reg Count', 'Size', 'Days', 'Assignment'])
Solution = Solution.loc[Solution['Assignment'] != 0]
Solution

,Course,Classroom,Time,Reg Count,Size,Days,Assignment
731,ACCT-370 14029,JFF LL105,F 14,130,149,1,1.0
3013,ACCT-370 14028,JKP202,M 14,37,54,2,1.0
3027,ACCT-370 14028,JKP202,W 14,37,54,2,1.0
3300,ACCT-370 14026,ACC303,T 14,46,46,2,1.0
3314,ACCT-370 14026,ACC303,H 14,46,46,2,1.0
5610,ACCT-370 14027,JFF240,T 14,47,48,2,1.0
5624,ACCT-370 14027,JFF240,H 14,47,48,2,1.0
7001,ACCT-371 14044,JFF LL105,M 10,128,149,1,1.0
9034,ACCT-371 14040,JFF331,M 16,25,36,2,1.0
9048,ACCT-371 14040,JFF331,W 16,25,36,2,1.0


In [17]:
# Concatenate day of week and time slot in each day
Time = []
for i in I:
    for j in J:
        Time.append(i+' '+str(j))

# Create a data frame with time as index and classroom as columns
ScheduleTable = pd.DataFrame(index = Time, columns = K)
# Assign blank value into each cell
ScheduleTable = ScheduleTable.fillna('')

# If a course is assigned to a specific time and classroom, fill it into corresponding cell in the data frame
for t in Time:
    for k in K:
        for a in Solution.index:
            if str(Solution.loc[a,'Time']) == str(t):
                if str(Solution.loc[a,'Classroom']) == str(k):
                    ScheduleTable.loc[t,k] = Solution.loc[a,'Course']
ScheduleTable

Room,ACC 306B,ACC201,ACC205,ACC236,ACC303,ACC306B,ACC310,ACC312,BRI202,BRI202A,...,JFF416,JFF417,JKP102,JKP104,JKP110,JKP112,JKP202,JKP204,JKP210,JKP212
M 8,,,FBE-529 15403,,,,,WRIT-340 66785,,BUAD-302 14681,...,,,,,,,,,,
M 10,ACCT-582 14289,BAEP-451 14378,BUAD-302 14662,MKT-599 16564,BUAD-281 14524,ACCT-574 14202,BUAD-281 14520,WRIT-340 66789,ACCT-373 14057,BUAD-304 14728,...,BUAD-497 15104,BUAD-306 14786,ACCT-470 14116,BAEP-451 14375,ECON-351 26358,ECON-352 26367,ACCT-430 14145,DSO-530 16276,FBE-391 15310,BUAD-311 14904
M 12,GSBA-612 16111,BUAD-497 15108,BUAD-302 14654,ACCT-473 14136,BUAD-497 15110,,ACCT-473 14137,WRIT-340 66734,ACCT-373 14056,BUAD-310 14892,...,MKT-556 16537,BUAD-302 14650,DSO-547 16280,ACCT-470 14115,MOR-588 16720,FBE-421 15324,BUAD-306 14782,ACCT-470 14117,BUAD-311T 14906,MOR-421 16677
M 14,,BUAD-281 14530,BAEP-554 14448,ACCT-581 14277,MKT-450 16496,,ACCT-372 14051,WRIT-340 66769,FBE-554 15425,BUAD-310 14897,...,BUAD-307 14848,BAEP-551 14446,BUAD-280 14509,BUAD-280 14507,FBE-421 15325,ECON-352 26368,ACCT-370 14028,BUAD-280 14513,FBE-441 15362,ECON-352 26363
M 16,,,DSO-582 16289,,,,,ACCT-574 14203,,BUAD-302 14677,...,,ACCT-581 14276,,,,,,,,
M 18,,,ACCT-580T 14273,,,,,WRIT-340 66779,,BUAD-302T 14704,...,,MOR-598 16730,,,,,,,,
M 20,,,BUAD-305 14768,,,,,WRIT-340 66716,,MKT-445 16493,...,,FBE-558 15440,,,,,,,,
T 8,,,ACCT-430 14146,,,,,WRIT-340 66781,,BUAD-302 14683,...,,ACCT-372 14052,,,,,,,,
T 10,,BUAD-281 14521,ACCT-371 14042,BUAD-306 14787,FBE-557 15436,BUAD-307 14822,ACCT-474 14140,WRIT-340 66719,MOR-463 16673,BUAD-304 14751,...,BUAD-497 15092,ACCT-416 14105,MKT-533 16530,DSO-510 16302,ECON-352 26365,BUAD-311 14912,ACCT-377 14067,FBE-535 15417,BUAD-311 14903,BUAD-311 14905
T 12,,BAEP-451 14379,ACCT-530 14207,MOR-431 16671,BUAD-281 14528,,ACCT-528 14242,BUAD-305 14766,ACCT-410 14003,BUAD-304 14755,...,MKT-405 16469,MOR-469 16680,DSO-570 16298,ECON-351 26349,ECON-351 26348,FBE-458 15367,BUAD-280 14508,ACCT-568T 14246,ACCT-474 14141,BUAD-302 14685


## Output several dataframes to the same excel file

In [18]:
writer = pd.ExcelWriter('output.xlsx')
Summary.to_excel(writer, sheet_name = 'Summary', index = False)
Solution.to_excel(writer, sheet_name = 'Solution', index = False)
ScheduleTable.to_excel(writer, sheet_name = 'Schedule Table')
writer.save()